# Deformable Registration 

This uses the SITK framework: Initiate the registration method object,set a metric for measuring image simialrity, set the type of transformation you want to use, set the iterative optimization method you want to use (e.g. Stochastic Gradient Descent) and run to find the transform. The oupute is an object with multiple information - one of which is the transformation. In this code it is stored in outTxParams = outTx.GetParameters() - here this is just the cooefiecients of the B-Spline Approximations.


See http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/, for more detailed information about the whole SITK package.

### IMPORTANT!!!
It is highly advised to perform the rigid transform before doing the the deforamble . Doing so produces better deformable registration, but also generates to images needed to perform the deformable registration. If you wish to perform the deformable only, please pre-process the images by eqaulizing the image histogram and if it is a high resolution image using a blur to denoise. 

Please provide the file_path to the hist-equalized images.

In [2]:
import numpy as np
import cv2
import matplotlib
from matplotlib import pyplot as plt
import SimpleITK as sitk
%matplotlib inline
from ipywidgets import widgets
from ipywidgets import fixed as fxd
import sys
import os
import tools

In [1]:
FOLDER_PATH = ''
FIXED_IMAGE_FILE_PATH = '' #Enter the MALDI optical image file path
MOVING_IMAGE_FILE_PATH = '' #Enter the HandE image file path
MOVING_IMAGE_FILE_PATH_NOHISTO = '' # This will be used when applying to transform to the non-histo-eqaulized image.

In [ ]:

fixed = sitk.ReadImage(FIXED_IMAGE_FILE_PATH, sitk.sitkFloat32)

moving = sitk.ReadImage(MOVING_IMAGE_FILE_PATH, sitk.sitkFloat32)

moving1 = sitk.ReadImage(MOVING_IMAGE_FILE_PATH_NOHISTO) # This will be used when applying to transform to the non-histo-eqaulized image.

def command_iteration(method) :
    print("{0:3} = {1:10.5f}".format(method.GetOptimizerIteration(),
                                     method.GetMetricValue()))
    print("\t#: ", len(method.GetOptimizerPosition()))


def command_multi_iteration(method) :
    print("--------- Resolution Changing ---------")



transformDomainMeshSize=[10]*moving.GetDimension()
tx = sitk.BSplineTransformInitializer(fixed,
                                      transformDomainMeshSize )

print("Initial Parameters:");
print(tx.GetParameters())

R = sitk.ImageRegistrationMethod()
R.SetMetricAsMattesMutualInformation(50)
R.SetOptimizerAsGradientDescentLineSearch(5.0, 100,
                                          convergenceMinimumValue=1e-4,
                                          convergenceWindowSize=5)
R.SetOptimizerScalesFromPhysicalShift( )
R.SetInitialTransform(tx)
R.SetInterpolator(sitk.sitkLinear)

R.SetShrinkFactorsPerLevel([6,2,1])
R.SetSmoothingSigmasPerLevel([6,2,1])

R.AddCommand( sitk.sitkIterationEvent, lambda: command_iteration(R) )
R.AddCommand( sitk.sitkMultiResolutionIterationEvent, lambda: command_multi_iteration(R) )

outTx = R.Execute(fixed, moving)

print("-------")
print(outTx)
print("Optimizer stop condition: {0}".format(R.GetOptimizerStopConditionDescription()))
print(" Iteration: {0}".format(R.GetOptimizerIteration()))
print(" Metric value: {0}".format(R.GetMetricValue()))

Initial Parameters:
(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [17]:
if ( not "SITK_NOSHOW" in os.environ ):

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed);
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(100)
    resampler.SetTransform(outTx)

    out = resampler.Execute(moving)
    out1 = resampler.Execute(moving1)
    simg1 = sitk.Cast(sitk.RescaleIntensity(fixed), sitk.sitkUInt8)
    simg2 = sitk.Cast(sitk.RescaleIntensity(out), sitk.sitkUInt8)
    cimg = sitk.Compose(simg1, simg2, simg1//2.+simg2//2.)
    sitk.WriteImage(out1, FOLDER_PATH + 'BSPLINEREgImage.png')
    sitk.WriteImage(cimg,FOLDER_PATH + 'BSPLINEOverLay.jpg')

In [ ]:
testTx = outTx


In [12]:
def functionss(s,txParams,aoutTx):
    newTx = aoutTx
    newTx.SetParameters(tuple(s*x for x in outTxParams))
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(fixed);
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetDefaultPixelValue(255)
    resampler.SetTransform(newTx)
    
    out = resampler.Execute(moving1) #Change to moving1 if you want the image output
    
    plt.figure(figsize=(12, 12))
    
    nda = sitk.GetArrayViewFromImage(out)
    plt.imshow(nda)
    plt.show()
    

    
outTxParams = outTx.GetParameters()
widgets.interact(functionss, s= (0,1.5,0.1),txParams = fxd(outTxParams),aoutTx = fxd(outTx))

<function __main__.functionss>